In [1]:
#-- Modules & Packages

import pandas as pd 
import numpy as np 

from glob import glob 
from time import strftime, sleep 
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay 

import yfinance as yf
yf.pdr_override()

#-- Plotting 

import seaborn as sns
import dash 
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc 
import plotly.express as px 
import dash_table 
import plotly.graph_objects as go 
from jupyter_dash import JupyterDash

C:\Users\hp\AppData\Local\Temp\ipykernel_10336\3459512367.py:20: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\hp\AppData\Local\Temp\ipykernel_10336\3459512367.py:21: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\hp\AppData\Local\Temp\ipykernel_10336\3459512367.py:24: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
# pip install jupyter_dash
# pip install dash_bootstrap_components

This Dashboard assumes one purchase at start date, start_x variable, and monitors the performance of the portfolio
since the acquisition of assets. I can build on top of this code by selecting from a transaction file, buy & sell dates,
buy & sell prices, fees, cumulative units, cumulative cost, average price of purchase, average selling price, etc...

Data Collecting and Saving.

In [3]:
#-- Cleaning columns string names
def clean_headers(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(','').str.replace(')','').str.replace(' ','_').str.replace('_/_','/')

#-- Getting timestamps for file names before saving 
def get_tmstmp():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

# last_file = glob('../inputs/transactions*.xlsx')[-1]
# print(last_file[-(len(last_file)) + (last_file.rfind('/')+1):])

# all_ops = pd.read_excel(last_file)
# all_ops.date = pd.to_datetime(all_ops.date, format = "%d/%m/%Y")

# all_tickers = list(all_ops['ticker'].unique())
blackList = ['PEPE-USD', 'DOGE-USD', 'ADA-USD']
# final_tickers = [tick for tick in all_tickers if tick not in blackList]
# print("Traded {} different cryptos").format(len(all_tickers))

# final_filtered = all_ops[~all_ops.ticker.isin(blackList)]

In [4]:
portfolio_tickers = ['BTC-USD', 'ETH-USD', 'ENA-USD', 'AVAX-USD']

one_year = datetime.today().year - 1
today = datetime.today()
start_x = datetime(2023, 1, 1)
end_x = today

start_assets = datetime(2023,1,1)
end_assets = today

start_ytd = datetime(one_year, 12, 31) + BDay(1)

def get_data(tickers, startDate, endDate):
    def data(ticker):
        data = (pdr.get_data_yahoo(ticker, start = startDate, end = endDate))
        data.columns = [x.lower() for x in data.columns]
        data.drop(columns = ['adj close'], axis = 1, inplace = True)
        return data
    datum = map(data, tickers)
    return(pd.concat(datum, keys = tickers, names = ['ticker', 'date']))

prices_df = get_data(portfolio_tickers, start_x, end_x)
prices_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


open          high           low         close  \
ticker   date                                                                 
BTC-USD  2023-01-01  16547.914062  16630.439453  16521.234375  16625.080078   
         2023-01-02  16625.509766  16759.343750  16572.228516  16688.470703   
         2023-01-03  16688.847656  16760.447266  16622.371094  16679.857422   
         2023-01-04  16680.205078  16964.585938  16667.763672  16863.238281   
         2023-01-05  16863.472656  16884.021484  16790.283203  16836.736328   
...                           ...           ...           ...           ...   
AVAX-USD 2024-04-09     49.883575     49.944057     46.394878     46.470196   
         2024-04-10     46.470482     47.967793     45.674950     47.311695   
         2024-04-11     47.311695     47.482182     45.679466     46.057598   
         2024-04-12     46.057159     46.870613     36.225365     39.417744   
         2024-04-13     39.417908     39.418114     29.727076     34.078411   

                          volume  
ticker   date                     
BTC-USD  2023-01-01   9244361700  
         2023-01-02  12097775227  
         2023-01-03  13903079207  
         2023-01-04  18421743322  
         2023-01-05  13692758566  
...                          ...  
AVAX-USD 2024-04-09    437892500  
         2024-04-10    530754231  
         2024-04-11    371204369  
         2024-04-12    966043351  
         2024-04-13   1507323529  

[1419 rows x 5 columns]

In [5]:
# tx_df = pd.read_excel(r"D:\_datasets\Projects_Coding\portfolio_dashboard_base2\transactions_finaldf_2021-05-15_12h19m_repo.xlsx")
# tx_df.ticker.unique()
# tx_df

In [6]:
portfolio_all_values = pd.DataFrame({'assetTicker': ['BTC-USD','ETH-USD', 'ENA-USD', 'AVAX-USD'], 
                                     'quantity':[500, 7500, 1000000,12000 ]})
portfolio_all_values


txs_prices = pd.merge(prices_df, portfolio_all_values, how = 'outer', left_index=True, right_index = True).fillna(0)




ValueError: cannot join with no overlapping index names